# Projeto final: [DS-PY-017] LÓGICA DE PROGRAMAÇÃO II (PY)
## Turma #970 - LM Tech Data Talents
Aluno: Renato Massamitsu Zama Inomata


O presente trabalho tem como objetivo criar um sistema de cadastro voltado à música. Suas funcionalidades foram divididas entre usuário comum e administrador.

A seguir, são as bibliotecas e funções externas que utilizei no programa.

In [7]:
import json
from functools import reduce
from IPython.display import clear_output


### Arquivos

Os arquivos são armazenados em formato `.json`. Optei por utilizar o formato para que eu me familiarizasse melhor com a estrutura de dicionários.

Os arquivos foram divididos em 3, `artistas.json`, `albuns.json` e `playlists.json`.


#### `artistas.json`
O arquivo `artistas.json` se trata de um dicionário de dicionários, onde a chave é o nome do artista e o valor um dicionário contendo `nome_album` e `id_album`. Ambas têm como valores uma tupla contendo respectivamente o nome e o ID do álbum. Optei por adicionar essa informação do `id_album` para me resguardar em casos que dois artistas possuam o mesmo nome para o álbum.

> Não previ o caso de termos artistas com nomes repetidos por ser razoavelmente raro. Para fins de entrega do trabalho não achei que fosse algo vital na funcionalidade do programa.
> > Dito isso, caso fosse implementado, provavelmente iria utilizar o `id_artista` como chave para o dicionário `artistas` e `nome_artista` como um dos elementos dentro do dicionário correspondente ao valor. 

In [8]:
#artistas
linkin_park = {"Linkin Park":   {"nome_album": ["A Thousand Suns", 
                                            "Meteora",
                                            "Minutes to Midnight",
                                            "LIVING THINGS",
                                            "Numb/Encore: MTV Ultimate Mash-Ups Presents Collision Course",
                                            "Hybrid Theory (Bonus Edition)"],
                                "id_album": [448,
                                            449,
                                            450,
                                            445,
                                            446,
                                            447]}}


#### `albuns.json`

O arquivo `albuns.json` também é um dicionário de dicionários, porém desta vez a chave é o `id_album` previamente mencionado. Os valores são `nome_album`, `nome_musica` e `id_musica`. A lógica para esses valores é equivalente, cada um recebe uma tupla com as informações respectivas.

In [9]:
#albuns
hybrid_theory = {"446": {"nome_album": "Hybrid Theory (Bonus Edition)",
                        "nome_musica": ["Crawling",
                                        "In the End",
                                        "One Step Closer"],
                        "id_musica": [1152,
                                    1153,
                                    1151]}}


#### `playlists.json`

Por fim, o arquivo `playlists.json` contém outro dicionário, dessa vez com o nome da playlist como chave e um dicionário contendo `nome_musica`, `id_musica` e `nome_artista` como seus valores. A lista `nome_artista` foi colocada nesse dicionário para facilitar as buscas por artista que o usuário poderia solicitar.

In [10]:
#playlist
random_playlist_07 = {"random_playlist#07": {"nome_musica": ["Topanga",
                                                            "Stir Fry",
                                                            "In the End",
                                                            "We're Almost There",
                                                            "Madness"],
                                            "id_musica": [1977, 
                                                        1372,
                                                        1151,
                                                        1342,
                                                        1410],
                                            "nome_artista": ["Trippie Redd",
                                                            "Migos",
                                                            "Linkin Park",
                                                            "Michael Jackson",
                                                            "Muse"]}}


Para facilitar meus testes com a funcionalidade do programa, busquei uma base de dados maior e encontrei este dataset: https://marianaossilva.github.io/DSW2019/index.html

Como não vimos pacotes mais específicos para trabalharmos com dados acabei utilizando o que já sabia de outras linguagens e programas para filtrar e organizar os dados em `.csv` disponibilizados no link anterior. 

Os dados no arquivo que encaminhei junto ao trabalho são todos os álbuns e músicas que encontrei no dataset para os artistas com a popularidade maior que 80. Foram 227 artistas, 833 álbuns e 2107 músicas que inclui nos arquivos.

Depois fiz um tratamento a parte para transformar os dados da tabela em dicionários, a fim de adequar o que eu havia proposto para as estruturas de artistas, álbuns e playlists.

Outra solução que eu poderia ter adotado seria armazenar os dados em `.csv` e lê-los como listas de listas, onde cada indexador da lista representasse uma informação diferente. Como queria praticar o uso de dicionários acabei transformando os dados de `.csv` em `.json` uma única vez (fora deste programa) e baseei meu código nesse formato.

### Menus

Busquei modularizar funções para que fossem chamadas numa função principal `main()`.

As funções `menu_principal()`, `menu_admin()`, `menu_user()`, `menu_buscar_playlist()` servem para imprimir o menu específico na tela, receber a opção do usuário através de um input e retornar a opção escolhida. 

> As opções, apesar de serem numeradas de 1 a 3, foram tratadas em forma de string por simplicidade.

Defini também a função `input_opcao()` para solicitar um input do usuário. A função irá rodar até que a opção digitada esteja dentro de uma lista `opcoes_validas` informada de acordo com seu uso.

In [11]:
def input_opcao(opcoes_validas):
    while True:
        opcao = input('Digite uma das opções: ')
        if opcao in opcoes_validas:
            return opcao
        
        print('Opção inválida.')

In [12]:
def menu_principal():
    print('------------------------')
    print('#### MENU PRINCIPAL ####')
    print('------------------------')
    print('[1] Logar como usuário')
    print('[2] Logar como admin')
    print('[3] Sair')
    opcao = input_opcao(['1', '2', '3'])
    return opcao

In [13]:
def menu_admin():
    print('------------------------')
    print('###### MENU ADMIN ######')
    print('------------------------')
    print('[1] Registrar artista')
    print('[2] Registrar álbum')
    print('[3] Sair')
    opcao = input_opcao(['1', '2', '3'])
    return opcao

In [14]:
def menu_user():
    print('------------------------')
    print('###### MENU USER #######')
    print('------------------------')
    print('[1] Buscar playlist')
    print('[2] Criar playlist')
    print('[3] Sair')
    opcao = input_opcao(['1', '2', '3'])
    return opcao

In [15]:
def menu_buscar_playlist():
    print('-----------------------')
    print('### BUSCAR PLAYLIST ###')
    print('-----------------------')
    print('[1] Buscar por música')
    print('[2] Buscar por artista')
    print('[3] Buscar por nome da playlist')
    opcao = input_opcao(['1', '2', '3'])
    return opcao

### Lendo arquivos

A princípio eu havia pensado no código para montar estruturas com tuplas. Porém, na hora de transformar o arquivo em JSON os dados acabam sendo formatados com `[]`. Considerando que pela funcionalidade do programa o usuário não deveria poder realizar manipulações dos dados, optei por manter a estrutura de listas.

#### `ler_arquivos`
O código abaixo serve para abrir os arquivos e realizar as leituras. Ele usa o `try` para primeiramente tentar abrir o arquivo. Caso o arquivo não seja encontrado ou apresente um `TypeError`, ele irá criar um arquivo vazio. Caso haja mais algum problema durante a execução, como por exemplo a transformação de JSON para dicionário não ocorra com sucesso, `ler_arquivos` apenas retornará um dicionário vazio.

In [16]:
def ler_arquivos(nome_arquivo: str) -> dict:
    try:
        with open(nome_arquivo, 'r') as arquivo:
            conteudo = arquivo.read()
            
        conteudo_json = json.loads(conteudo)

        return conteudo_json
    
    except (FileNotFoundError, TypeError):
        with open(nome_arquivo, 'w') as arquivo:
            conteudo = arquivo.write('{}')
        
        return dict()

    except:
        return dict()

#### `int_id_album`
A função `int_id_album` transforma as chaves do dicionário `albuns` em inteiros. Tive que criá-la pois estes dados estavam sendo armazenados como strings nas chaves quando utilizei a função `json.dumps()`.

In [17]:
def int_id_album(albuns: dict) -> dict:

    return {int(album_id): album for album_id, album in albuns.items()}

#### `ler_tudo`
`ler_tudo` lê os arquivos de artistas, álbuns e playlists e no caso dos álbuns, também faz a conversão para inteiros das chaves.

In [18]:
def ler_tudo():
    artistas = ler_arquivos('artistas.json')
    albuns = ler_arquivos('albuns.json')
    playlists = ler_arquivos('playlists.json')

    albuns = int_id_album(albuns)

    return artistas, albuns, playlists

#### `escrever`
`escrever` é a função que irá escrever os arquivos toda vez que solicitado. Optei por utilizar `'w'` como parâmetro do `open` pois eu havia restringido a leitura na função `ler_arquivos`, assim `'r+'` não parecia ser necessário. Também não queria trabalhar com `'a'` pois geraria uma complexidade na hora de inserir os dados em formato JSON para os arquivos.

In [42]:
def escrever(conteudo: dict, nome_arquivo: str) -> None:
    try:
        with open(nome_arquivo, 'w') as arquivo:
            arquivo.write(json.dumps(conteudo, indent = 4))
    except Exception as e:
        print(f'Erro: {e}')
        
    return None

### Funções auxiliares


#### `max_ids()`

A ideia de utilizar IDs para individualizar os álbuns e músicas, independentemente se houverem nomes repetidos, gerou a problemática em como definir IDs para cadastros novos.

Adotei que os IDs seriam incrementados de um em um, conforme cadastros novos fossem realizados. Dessa maneira, bastaria eu determinar qual é o maior ID tanto para `id_album` quanto para `id_musica` e então incrementá-los para obter o ID do próximo cadastro.

Esse inclusive é o motivo de eu ter utilizado `int` como o tipo de variável para essas informações, para que fosse possível realizar operações matemáticas com os IDs.

Não achei interessante utilizar o `len` dos objetos pois se por algum motivo houvesse algum número que fosse "pulado" nos IDs, eu teria problema de IDs repetidos na estrutura.

In [20]:
def max_ids(albuns):
    if albuns == {}:
        return 0, 0
    
    max_id_album = max(albuns.keys())
    max_id_musica = max([valor['id_musica'] if type(valor['id_musica']) == int else max(valor['id_musica']) for valor in albuns.values()])

    return max_id_album, max_id_musica

### Funções de cadastro
As funções `cadastrar_artista()`, `cadastrar_album()`, `cadastrar_playlist()`, conforme próprio nome sugere, servem para realizar cadastro.




 

#### `cadastrar_artista`

O programa pede para o usuário digitar o nome para o novo artista, se ele já existir na base de dados ele retorna uma mensagem dizendo que o artista já foi cadastrado. Caso contrário, ele gera uma estrutura vazia com a chave `nome_artista`.

In [21]:
def cadastrar_artista(artistas):
    novo_artistas = artistas.copy()
    artista = input('Digite um novo artista: ')

    if artista not in artistas:
        novo_artistas[artista] = {'nome_album': [],
                                    'id_album': []}
        return novo_artistas
    
    print('Artista já cadastrado.')
    return novo_artistas

#### `cadastrar_album`

Nessa função o usuário digita primeiramente o nome do artista, caso ele não seja encontrado é printado uma mensagem de erro. 

Em seguida, solicita-se o nome para o álbum, da mesma forma, caso ele já exista, uma mensagem de erro aparece.

Depois, é solicitado o número de músicas e finalmente cada música é solicitada uma a uma de acordo com o número previamente informado.

O programa finaliza realizando appends de `nome_album` e `id_album` no dicionário de artistas e também criando o novo álbum no dicionário de álbuns.

In [33]:
def cadastrar_album(artistas, albuns, max_id_album, max_id_musica):
    novo_artistas = artistas.copy()
    novo_albuns = albuns.copy()
    novo_max_id_album = max_id_album
    novo_max_id_musica = max_id_musica

    artista = input('Digite o nome do artista: ')
    if artista not in artistas:
        print('Artista não encontrado. Cancelando a operação.')
        return novo_artistas, novo_albuns, novo_max_id_album, novo_max_id_musica
    
    novo_album = input('Digite o nome do álbum: ')
    
    if novo_album in novo_artistas[artista]['nome_album']:
        print('Álbum já existe. Cancelando a operação.')
        return novo_artistas, novo_albuns, novo_max_id_album, novo_max_id_musica

    try:
        num_musicas = int(input('Digite o número de músicas no álbum: '))
    except:
        print('Número de músicas inválido. Cancelando a operação.')
        return novo_artistas, novo_albuns, novo_max_id_album, novo_max_id_musica
    
    nome_musicas = [input(f'Digite o nome da música {i+1}: ') for i in range(num_musicas)]
    id_musicas = [id for id in range(novo_max_id_musica+1, novo_max_id_musica+1+num_musicas)]

    novo_max_id_album += 1
    novo_max_id_musica += num_musicas

    novo_artistas[artista]['nome_album'].append(novo_album)
    novo_artistas[artista]['id_album'].append(novo_max_id_album)

    novo_albuns[novo_max_id_album] = {'nome_album': novo_album,
                                      'nome_musica': nome_musicas,
                                      'id_musica': id_musicas}
    
    return novo_artistas, novo_albuns, novo_max_id_album, novo_max_id_musica

#### `escolher_musica`
A função `escolher_musica` a seguir foi criada para auxiliar na função `cadastrar_playlist`. É ela quem irá receber os inputs do usuário para que o mesmo escolha a música a ser adicionada a playlist.

In [48]:
def escolher_musica(artistas, albuns):
    print('Confira a lista de artistas:')
    [print(artista) for artista in sorted(artistas)]
    
    nome_artista = input('Digite o nome do artista: ')
    while nome_artista not in artistas:
        nome_artista = input('Artista não encontrado. Digite o nome do artista')
    
    ids_albuns = artistas[nome_artista]['id_album']

    clear_output(wait=True)
    print('-'*64)
    print(f'Confira os álbuns do artista {nome_artista}:')
    albuns_artista = [infos['nome_album'] for album, infos in albuns.items() if album in ids_albuns]
    [print(album) for album in albuns_artista]
    
    nome_album = input('Digite o nome do álbum: ')
    while nome_album not in albuns_artista:
        nome_album = input('Álbum não encontrado. Digite o nome do álbum: ')
    
    id_album = [id for album, id in zip(artistas[nome_artista]['nome_album'], artistas[nome_artista]['id_album']) if nome_album == album][0]

    clear_output(wait=True)
    print('-'*64)
    print(f'Confira as músicas do artista {nome_artista} no álbum {nome_album}: ')
    musicas_album = albuns[id_album]['nome_musica']
    [print(musica) for musica in musicas_album]
    
    nome_musica = input('Digite o nome da música: ')
    while nome_musica not in musicas_album:
        nome_musica = input('Música não encontrada. Digite o nome da música: ')
    
    id_musica = [id for musica, id in zip(albuns[id_album]['nome_musica'], albuns[id_album]['id_musica']) if nome_musica == musica][0]

    return nome_artista, nome_album, id_album, nome_musica, id_musica

#### `cadastrar_playlist`
A função `cadastrar_playlist` primeiro valida se os dicionários `albuns` ou `artistas` estão vazios.

Em seguida solicita o nome para a playlist, repetindo até que o usuário digite um nome que já está sendo utilizado.

Por fim, ela vai solicitando em loop `while True` a escolha das músicas com a função anterior. Ao final é solicitado que o usuário digite `S` ou `N` para que ele continue ou não a adicionar mais músicas.


In [24]:
def cadastrar_playlist(artistas, albuns, playlists):

    if albuns == {} or artistas == {}:
        print('Parece que não há dados de artistas e álbuns cadastrados, por favor consulte o seu administrador.')
        return None

    nome_playlist = input('Informe o nome para a sua playlist: ')
    while nome_playlist in playlists:
        nome_playlist = input('Nome já cadastrado. Informe o nome para a sua playlist: ')
    
    playlist_musicas = []
    playlist_ids = []
    playlist_artistas = []
    playlists_novas = playlists.copy()

    while True:
        nome_artista, *_,  nome_musica, id_musica = escolher_musica(artistas, albuns)
        playlist_musicas.append(nome_musica)
        playlist_ids.append(id_musica)
        playlist_artistas.append(nome_artista)

        opcao = input('Deseja continuar? (S/N) ').upper()
        while opcao != 'S' and opcao != 'N':
            opcao = input('Opção inválida. Deseja continuar? (S/N) ').upper()
            
        if opcao == 'N':
            playlists_novas[nome_playlist] = {'nome_musica': playlist_musicas,
                                        'id_musica': playlist_ids,
                                        'nome_artista': playlist_artistas}

            return playlists_novas

### Buscas

#### `filter_conteudo`
A função `filter_conteudo` foi criada para filtrar os conteúdos de um dicionário passando os pares chave/valor dentro de um argumento `**kwargs`.

Ela foi utilizada durante a execução das funções de buscas de playlists. A princípio, eu havia previsto que as próprias funções de buscas fariam sua própria filtragem, porém consegui desenvolver essa função de forma bastante genérica e aplicável para diversos casos. 

Apesar de usar conteúdos bastante específicos e consideravelmente mais difíceis deste módulo, `filter_conteudo` não foi uma forma de aplicar todo o "restante" da matéria de uma só vez. Como eu queria que ela fosse bastante versátil, pensei em criar os `**kwargs`, uma vez que a minha estrutura de dados permitiria facilmente trabalhar com este tipo de argumento (todos os dicionários são, no máximo, um dicionário de dicionário e não dicionários de dicionários de dicionários).

A linha

> keys = [[chave for chave, conteudo in dicionario.items() if filtro in conteudo[parametro]] for parametro, filtro in kwargs.items()]

foi a forma que eu encontrei de utilizar uma compreensão de listas para filtrar o conteúdo do dicionário. Ela percorre os parâmetros de `kwargs` e depois os elementos do dicionário, gerando uma lista de listas. 

A primeira versão desse código não estava prevendo a utilização de mais de um `kwarg`, não sendo necessário um loop específico para isso. Apesar de realmente não utilizar mais de um `kwarg` durante a execução do meu programa, eu pensei que talvez poderia aplicar esse tipo de filtragem em algum outro momento. 

Eu cogitei também transformar essa compreensão de listas em um loop for, porém achei a compreensão de lista mais interessante uma vez que ela gera uma lista, para cada parâmetro passado na função, que por sua vez contém uma lista de ocorrências desse parâmetro no dicionário. Essa lista de listas é importante pois com o argumento `match_exato` eu posso definir se quero que o resultado da função seja as ocorrências que correspondem a exatamente todos os parâmetros ou se elas correspondem a pelo menos um dos parâmetros passados.

A ideia é que com a lista de listas, eu pude transformar as listas internas em sets e compará-las com as demais. Dessa forma, se `match_exato` for `False` eu estaria realizando a união dos conjuntos, enquanto que se fosse `True` o programa estaria realizando a interseção dos mesmos. Neste contexto, fez bastante sentido também utilizar `reduce`, pois eu queria aplicar a transformação em set, a união/interseção deles e ir acumulando esse resultado ao percorrer a lista.

Finalmente, `nome_playlist` foi adicionado para o caso específico das buscas por nome da playlist, uma vez que ela se trata da chave do dicionário e portanto não necessitaria de todo o conteúdo aplicado para os demais casos.

In [25]:
def filter_conteudo(dicionario, nome_playlist = None, match_exato = False, **kwargs):
    if nome_playlist != None:
        keys = [chave for chave in dicionario.keys() if chave == nome_playlist]
        return keys
    
    else:
        # para cada parametro em **kwargs, verificar se kwargs está em um conteúdo do dicionário, se estiver retornar a chave
        keys = [[chave for chave, conteudo in dicionario.items() if filtro in conteudo[parametro]] for parametro, filtro in kwargs.items()]
        
        # keys é uma lista que contém listas com todos as chaves que eram iguais aos valores de kwargs, para cada parametro de kwargs

        # se match_exato == False, quer dizer que iremos fazer a união entre as listas dentro da lista keys
        # caso match_exato == True, faremos a interseção das listas de keys

        if match_exato == False:
            keys = list(reduce(lambda x, y: set(x).union(set(y)), keys))
    
        else:
            keys = list(reduce(lambda x, y: set(x).intersection(set(y)), keys))
    
        return keys

In [ ]:
# Teste filter_conteudo

artistas, albuns, playlists = ler_tudo()

a = filter_conteudo(playlists, match_exato = False, nome_artista = 'Michael Jackson', nome_musica = 'In the End')
print(a)
b = filter_conteudo(playlists, match_exato = True, nome_artista = 'Michael Jackson', nome_musica = 'In the End')
print(b)

#### `print_playlists`

`print_playlists` é uma função auxiliar para printar na tela os resultados encontrados nas buscas por playlists.

In [27]:
def print_playlists(playlists:dict, nomes_playlists:list, nome_artista:str = None, nome_musica:str = None, nome_playlist:str = None) -> None:

    if nomes_playlists == []:
        print('Não encontramos playlists com os dados informados :(')
        return None

    if nome_artista != None:
        print(f'Playlists contendo [{nome_artista}]')
    
    if nome_musica != None:
        print(f'Playlists contendo [{nome_musica}]')

    if nome_playlist != None:
        print(f'Playlist [{nome_playlist}]')

    for playlist in nomes_playlists:
        print(f'\n>> {playlist} <<\n')
        [print(f'[{id}] {musica[1]} | {musica[0]}') for id, musica in enumerate(zip(playlists[playlist]['nome_musica'], playlists[playlist]['nome_artista']))]
        print('-'*64)
    return None

#### `buscar_playlist_musica`, `buscar_playlist_artista` e `buscar_playlist_nome`
As funções de busca a seguir são as que recebem o input do usuário e chamam as funções anteriores (`filter_conteudo` e `print_playlist`) para filtragem e impressão na tela.

In [28]:
def buscar_playlist_musica(playlists):
    nome_musica = input('Digite o nome da música: ')

    filter_playlist = filter_conteudo(playlists, nome_musica = nome_musica)

    print_playlists(playlists, filter_playlist, nome_musica = nome_musica)

    return filter_playlist

In [29]:
def buscar_playlist_artista(playlists):
    nome_artista = input('Digite o nome do artista: ')

    filter_playlist = filter_conteudo(playlists, nome_artista = nome_artista)

    print_playlists(playlists, filter_playlist, nome_artista = nome_artista)

    return filter_playlist

In [30]:
def buscar_playlist_nome(playlists):
    nome_playlist = input('Digite o nome da playlist: ')

    filter_playlist = filter_conteudo(playlists, nome_playlist = nome_playlist)

    print_playlists(playlists, filter_playlist, nome_playlist = nome_playlist)

    return filter_playlist

### Código principal

A seguir, o código principal do programa.

O programa lê os arquivos JSON uma vez ao iniciar ao programa e também antes de cada etapa em que é solicitado um dos três dicionários principais.

Da mesma forma, toda vez que o usuário realiza uma operação de cadastro, o programa irá escrever o JSON do dicionário cadastrado.

In [39]:
import json
from functools import reduce
from IPython.display import clear_output

def main():

    print('Bem-vindo(a) ao Vin Deezer!')

    # Inicializar os arquivos
    artistas, albuns, playlists = ler_tudo()

    while True:
        max_id_album, max_id_musica = max_ids(albuns)

        opcao = menu_principal()
        clear_output(wait=True)
        
        # Menu usuário
        if opcao == '1':
            while True:
                opcao = menu_user()

                if opcao == '1':
                    clear_output(wait=True)
                    print('------------------------')
                    print('Buscar playlist')
                    opcao = menu_buscar_playlist()

                    if opcao == '1':                        
                        clear_output(wait=True)
                        print('------------------------')
                        print('Buscar por música')
                        playlists = ler_arquivos('playlists.json')
                        buscar_playlist_musica(playlists)

                    elif opcao == '2':                    
                        clear_output(wait=True)    
                        print('------------------------')
                        print('Buscar por artista')
                        playlists = ler_arquivos('playlists.json')
                        buscar_playlist_artista(playlists)

                    elif opcao == '3':
                        clear_output(wait=True)
                        print('------------------------')
                        print('Buscar por nome da playlist')
                        playlists = ler_arquivos('playlists.json')
                        buscar_playlist_nome(playlists)


                elif opcao == '2':
                    clear_output(wait=True)
                    print('------------------------')
                    print('Criar playlist')
                    artistas, albuns, playlists = ler_tudo()
                    playlists = cadastrar_playlist(artistas, albuns, playlists)
                    escrever(playlists, 'playlists.json')

                else:
                    clear_output(wait=True)
                    print('------------------------')
                    print('Saindo.')
                    break
        
        # Menu admin
        elif opcao == '2':
            while True:
                opcao = menu_admin()

                if opcao == '1':
                    clear_output(wait=True)
                    print('------------------------')
                    print('Registrar artista')
                    artistas = ler_arquivos('artistas.json')
                    artistas = cadastrar_artista(artistas)
                    escrever(artistas, 'artistas.json')

                elif opcao == '2':
                    clear_output(wait=True)
                    print('------------------------')
                    print('Registrar álbum')
                    artistas, albuns, playlists = ler_tudo()
                    artistas, albuns, max_id_album, max_id_musica = cadastrar_album(artistas, albuns, max_id_album, max_id_musica)
                    escrever(artistas, 'artistas.json')
                    escrever(albuns, 'albuns.json')
                    
                else:
                    clear_output(wait=True)
                    print('------------------------')
                    print('Saindo.')
                    break
        
        # Sair
        else:
            print('Encerrando o programa! Obrigado por utilizá-lo :)')
            break

In [40]:
main()

Encerrando o programa! Obrigado por utilizá-lo :)
